In [1]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
from scipy import stats

In [2]:
df=pd.read_csv("numerai_training_data.csv")
df.head()
df2=pd.read_csv('numerai_tournament_data.csv')
df2.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.0,0.75,0.5,...,0.75,0.75,1.00,0.75,0.50,0.5,1.0,0.00,0.00,0.25
1,n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.5,0.00,0.0,...,0.50,0.50,0.75,1.00,0.75,0.5,0.5,0.50,0.50,0.50
2,n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.0,1.00,1.0,...,0.00,0.00,0.50,0.25,0.00,0.0,0.5,0.50,0.00,1.00
3,n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.0,0.00,0.0,...,0.00,0.00,0.00,0.25,0.00,0.0,0.0,0.25,0.25,0.50
4,n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.0,0.50,0.5,...,0.50,0.75,0.00,0.00,0.75,1.0,0.0,0.25,1.00,0.75


In [3]:
features = [c for c in df if c.startswith("feature")]
X = df[features]

In [4]:
def ar1(x):
    return np.corrcoef(x[:-1], x[1:])[0,1]

def autocorr_penalty(x):
    n = len(x)
    p = ar1(x)
    return np.sqrt(1 + 2*np.sum([((n - i)/n)*p**i for i in range(1,n)]))

def smart_sharpe(x):
    return np.mean(x)/(np.std(x, ddof=1)*autocorr_penalty(x))

import matplotlib.pyplot as plt
def era_boost_train(X, y, era_col, proportion=0.5, trees_per_step=10, num_iters=200):
    model = GradientBoostingRegressor(max_depth=5, learning_rate=0.01, max_features="sqrt", subsample=0.5, n_estimators=trees_per_step, warm_start=(num_iters>1))
    features = X.columns
    model.fit(X, y)
    new_df = X.copy()
    new_df["target"] = y
    new_df["era"] = era_col
    for i in range(num_iters-1):
        print(f"iteration {i}")
        # score each era
        print("predicting on train")
        preds = model.predict(X)
        new_df["pred"] = preds
        era_scores = pd.Series(index=new_df["era"].unique())
        print("getting per era scores")
        for era in new_df["era"].unique():
            era_df = new_df[new_df["era"] == era]
            era_scores[era] = stats.spearmanr(era_df["pred"], era_df["target"])[0]
        era_scores.sort_values(inplace=True)
        worst_eras = era_scores[era_scores <= era_scores.quantile(proportion)].index
        print(list(worst_eras))
        worst_df = new_df[new_df["era"].isin(worst_eras)]
        era_scores.sort_index(inplace=True)
        era_scores.plot(kind="bar")
        print("performance over time")
        plt.show()
        print("autocorrelation")
        print(ar1(era_scores))
        print("mean correlation")
        print(np.mean(era_scores))
        print("standard deviation")
        print(np.std(era_scores))
        print("sharpe")
        print(np.mean(era_scores)/np.std(era_scores))
        print("smart sharpe")
        print(smart_sharpe(era_scores))
        model.n_estimators += trees_per_step
        print("fitting on worst eras")
        model.fit(worst_df[features], worst_df["target"])
    return model

boost_model = era_boost_train(X, df["target"], era_col=df["era"], proportion=0.5, trees_per_step=10, num_iters=20)

iteration 0
predicting on train


C:\Users\yashd\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


getting per era scores
['era68', 'era103', 'era58', 'era69', 'era91', 'era104', 'era60', 'era41', 'era67', 'era85', 'era42', 'era73', 'era9', 'era107', 'era26', 'era112', 'era117', 'era100', 'era27', 'era110', 'era113', 'era66', 'era80', 'era46', 'era31', 'era33', 'era7', 'era74', 'era62', 'era32', 'era14', 'era119', 'era2', 'era3', 'era50', 'era84', 'era82', 'era18', 'era13', 'era75', 'era111', 'era89', 'era55', 'era65', 'era102', 'era87', 'era19', 'era71', 'era114', 'era11', 'era101', 'era106', 'era40', 'era118', 'era21', 'era29', 'era57', 'era79', 'era77', 'era54']
performance over time


<Figure size 640x480 with 1 Axes>

autocorrelation
0.09746888081349542
mean correlation
0.05654293032324128
standard deviation
0.02855021668576949
sharpe
1.980472896075231
smart sharpe
1.789959372105177
fitting on worst eras
iteration 1
predicting on train


...

autocorrelation
-0.014009247069327204
mean correlation
0.09134088800006895
standard deviation
0.0034413025790958505
sharpe
26.542533212545166
smart sharpe
26.80149739795898
fitting on worst eras
iteration 18
predicting on train


C:\Users\yashd\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


getting per era scores
['era91', 'era103', 'era97', 'era100', 'era84', 'era44', 'era119', 'era32', 'era68', 'era66', 'era101', 'era3', 'era28', 'era50', 'era41', 'era40', 'era114', 'era45', 'era116', 'era43', 'era13', 'era73', 'era99', 'era33', 'era55', 'era110', 'era94', 'era54', 'era57', 'era113', 'era118', 'era35', 'era22', 'era107', 'era105', 'era72', 'era115', 'era71', 'era19', 'era98', 'era6', 'era69', 'era85', 'era49', 'era81', 'era27', 'era48', 'era65', 'era74', 'era117', 'era79', 'era30', 'era25', 'era62', 'era53', 'era8', 'era106', 'era7', 'era31', 'era93']
performance over time


<Figure size 640x480 with 1 Axes>

autocorrelation
-0.09614282550600864
mean correlation
0.09218776127660235
standard deviation
0.003165104650555621
sharpe
29.12629168846571
smart sharpe
31.91543771512601
fitting on worst eras


In [14]:
predictions = pd.Series(boost_model.predict(df2[features]))
submission = pd.concat([df2['id'],predictions],axis=1)
submission

,id,0
0,n0003aa52cab36c2,0.496328
1,n000920ed083903f,0.498784
2,n0038e640522c4a6,0.509976
3,n004ac94a87dc54b,0.500018
4,n0052fe97ea0c05f,0.499111
...,...,...
1709330,nffe60faae42d3c2,0.501668
1709331,nffe9b02e07ab272,0.504702
1709332,nffe9f5eb263211c,0.500847
1709333,nfff507dae8c8e6c,0.500472


In [15]:
submission.columns = ['id','prediction']

In [16]:
submission

,id,prediction
0,n0003aa52cab36c2,0.496328
1,n000920ed083903f,0.498784
2,n0038e640522c4a6,0.509976
3,n004ac94a87dc54b,0.500018
4,n0052fe97ea0c05f,0.499111
...,...,...
1709330,nffe60faae42d3c2,0.501668
1709331,nffe9b02e07ab272,0.504702
1709332,nffe9f5eb263211c,0.500847
1709333,nfff507dae8c8e6c,0.500472


In [18]:
submission.to_csv("predictions.csv",index=False)